In [1]:
import pandas as pd
import networkx as nx
import argparse
import itertools

In [2]:
def add_weigths(t, Gr):
    for i in range(len(t)):
        for j in range(i+1, len(t)):
            Gr.edges[t[i], t[j]]['weight'] += 1

In [3]:
sdf = pd.read_csv('./data/reddit/subreddits.csv')
subreddits = sdf.values.tolist()

In [4]:
years = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']

In [10]:
df0 = pd.DataFrame()
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
df5 = pd.DataFrame()
df6 = pd.DataFrame()
df7 = pd.DataFrame()
df8 = pd.DataFrame()
df9 = pd.DataFrame()

In [11]:
def mask_year(year, temp_df):
    mask=temp_df["Publish Date"].str.contains(year,na=False)
    temp_au = pd.DataFrame(temp_df[mask].groupby('Author')['Score'].sum())
    temp_au['sub'] = sub + ' '
    return temp_au

In [16]:
DataFrames = []

In [17]:
for s in subreddits:
    sub = str(s)[5:-5]
    temp_df = pd.read_csv(f'./data/reddit/cm/{sub}_comments.csv')
    temp_df = temp_df[temp_df.Author != '[deleted]']
    
    if s == subreddits[0]:
        for i in range(10):
            DataFrames.append(mask_year(years[i], temp_df))
    else:
        for i in range(10):
            DataFrames[i] = DataFrames[i].append(mask_year(years[i], temp_df))

In [25]:
relations = []

In [26]:
for i in range(10):
    relations.append(pd.DataFrame(DataFrames[i].groupby('Author')['sub'].sum()))

In [28]:
relations[5].head()

,sub
Author,
---__--_---__---_-__,TheRedPill
--CellarDoor--,TheBluePill
--Cherry--,TheRedPill
--Edog--,TheRedPill
--TakeThePill,TheRedPill asktrp


In [29]:
Graphs = []

In [30]:
for i in range(10):
    Graphs.append(nx.Graph())
                                                            
for s in subreddits:                                    
    sub = str(s)[5:-5]                                  
    for i in range(10):
        Graphs[i].add_node(sub)
                                                            
t = [str(s)[5:-5] for s in subreddits]        
for i in range(10):
    Graphs[i].add_weighted_edges_from(itertools.product(t, t, [0])) 

In [31]:
for i in range(10):
    for Author in relations[i]['sub']: 
        sub = Author.split()       
        if len(sub) > 1:           
            add_weigths(sub, Graphs[i])    

In [34]:
for i in range(10):
    nx.write_gexf(Graphs[i], f"./data/reddit/graphs/graph_201{i}.gexf")       